In [13]:
from my_layers import *
import numpy as np
from sklearn.decomposition import NMF

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
V = np.load('synthetic_data/x.syn.many.types.0.5_sp.sp.npy')

In [15]:
# Build training data
model_1 = NMF(n_components=22, solver='mu', beta_loss='frobenius', verbose=True)
W = model_1.fit_transform(V)
H = model_1.components_

Epoch 10 reached after 0.018 seconds, error: 33242.882453
Epoch 20 reached after 0.027 seconds, error: 30221.810710
Epoch 30 reached after 0.036 seconds, error: 29102.570433
Epoch 40 reached after 0.045 seconds, error: 28374.046578
Epoch 50 reached after 0.054 seconds, error: 27852.524232
Epoch 60 reached after 0.063 seconds, error: 27563.763205
Epoch 70 reached after 0.071 seconds, error: 27371.322122
Epoch 80 reached after 0.079 seconds, error: 27213.830440
Epoch 90 reached after 0.087 seconds, error: 27059.942165
Epoch 100 reached after 0.099 seconds, error: 26891.908131
Epoch 110 reached after 0.106 seconds, error: 26715.360412
Epoch 120 reached after 0.113 seconds, error: 26559.299798
Epoch 130 reached after 0.121 seconds, error: 26458.546990
Epoch 140 reached after 0.128 seconds, error: 26381.294501
Epoch 150 reached after 0.135 seconds, error: 26302.755394
Epoch 160 reached after 0.142 seconds, error: 26232.160219
Epoch 170 reached after 0.149 seconds, error: 26171.769360
Epoch 

In [16]:
M = torch.from_numpy(V)

In [5]:
M.shape

torch.Size([96, 1350])

In [12]:
M.device

device(type='cpu')

In [6]:
V.shape

(96, 1350)

In [17]:
clipper = WeightClipper()

In [18]:
relu_layer = MuReluLayer()
print(relu_layer)

MuReluLayer(
  (fc1): Linear(in_features=22, out_features=22, bias=False)
  (fc2): Linear(in_features=96, out_features=22, bias=False)
)


In [11]:
h = torch.rand(comp).float()
input = (h.float(),M[:,3].float())
out = relu_layer(*input)
print(out)

tensor([   0.0000,    0.0000,  430.4540,  191.2160,  354.4443,    0.0000,
           0.0000,    0.0000,  582.4852,  332.3528, 1833.3652,  333.9752,
         151.2865,    0.0000,    0.0000, 1047.6648,    0.0000,    0.0000,
          85.8514,   75.6119,    0.0000,  106.2368], grad_fn=<ReluBackward0>)


In [0]:
# for name, param in nmf.named_parameters():
#   if param.requires_grad:
#     print (name, param.data)

In [22]:
input = (h.float(),M[:,3].float())
relu_net = MuReluNet()
print(relu_net)
relu_net.apply(clipper)
out = relu_net(*input)
print(out)

MuReluNet(
  (layer1): MuReluLayer(
    (fc1): Linear(in_features=22, out_features=22, bias=False)
    (fc2): Linear(in_features=96, out_features=22, bias=False)
  )
  (layer2): MuReluLayer(
    (fc1): Linear(in_features=22, out_features=22, bias=False)
    (fc2): Linear(in_features=96, out_features=22, bias=False)
  )
  (layer3): MuReluLayer(
    (fc1): Linear(in_features=22, out_features=22, bias=False)
    (fc2): Linear(in_features=96, out_features=22, bias=False)
  )
  (layer4): MuReluLayer(
    (fc1): Linear(in_features=22, out_features=22, bias=False)
    (fc2): Linear(in_features=96, out_features=22, bias=False)
  )
  (layer5): MuReluLayer(
    (fc1): Linear(in_features=22, out_features=22, bias=False)
    (fc2): Linear(in_features=96, out_features=22, bias=False)
  )
)
tensor([ 436.1946,  124.8559,  727.6868,  166.7366,  444.6982,   71.1719,
         779.2117, 3011.0420,  668.7136,  147.0430,  241.6931,   90.8705,
         146.8523,  153.2276,   89.6422,  791.5676,  159.9097,  

https://github.com/pytorch/pytorch/issues/19808#issuecomment-487257761

In [0]:
import torch.optim as optim

criterion = nn.MSELoss()
optimizerSGD = optim.SGD(MuReluNet().parameters(), lr=0.0000001)
optimizerADAM = optim.Adam(MuReluNet().parameters(), lr=1e-5)

In [0]:
import torch.utils.data as data_utils
h_init = torch.rand(samples, comp).float()
v = torch.from_numpy(V.T).float()
labels = torch.from_numpy(H.T).float()
dataset = data_utils.TensorDataset(h_init, v, labels)

In [0]:
trainloader = torch.utils.data.DataLoader(dataset, batch_size=4,shuffle=True, num_workers=2)

In [17]:
net = MuReluNet()
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        h_i, vi, labels = data
        inputs = (h_i, vi)
        # zero the parameter gradients
        optimizerADAM.zero_grad()

        # forward + backward + optimize
        # net.apply(clipper)
        outputs = net(*inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizerADAM.step()

      
        # print statistics
        running_loss += loss.item()
        if i % 30 == 29:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 30))
            running_loss = 0.0

print('Finished Training')

[1,    30] loss: nan
[1,    60] loss: nan
[1,    90] loss: nan
[1,   120] loss: nan
[1,   150] loss: nan
[1,   180] loss: nan
[1,   210] loss: nan
[1,   240] loss: nan
[1,   270] loss: nan
[1,   300] loss: nan
[1,   330] loss: nan
[2,    30] loss: nan
[2,    60] loss: nan
[2,    90] loss: nan
[2,   120] loss: nan
[2,   150] loss: nan
[2,   180] loss: nan
[2,   210] loss: nan
[2,   240] loss: nan
[2,   270] loss: nan
[2,   300] loss: nan
[2,   330] loss: nan
Finished Training


In [23]:
net = IstaNet()
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        h_i, vi, labels = data
        inputs = (h_i, vi)
        # zero the parameter gradients
        optimizerADAM.zero_grad()

        # forward + backward + optimize
        # net.apply(clipper)
        outputs = net(*inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizerADAM.step()

      
        # print statistics
        running_loss += loss.item()
        if i % 30 == 29:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 30))
            running_loss = 0.0

print('Finished Training')

[1,    30] loss: 82722.866
[1,    60] loss: 71299.042
[1,    90] loss: 83622.511
[1,   120] loss: 260870.381
[1,   150] loss: 49018.992
[1,   180] loss: 312221.886
[1,   210] loss: 101149.437
[1,   240] loss: 114237.703
[1,   270] loss: 83094.729
[1,   300] loss: 107012.277
[1,   330] loss: 44719.749
[2,    30] loss: 104694.394
[2,    60] loss: 309168.424
[2,    90] loss: 115944.783
[2,   120] loss: 80325.287
[2,   150] loss: 40091.489
[2,   180] loss: 54033.920
[2,   210] loss: 204193.557
[2,   240] loss: 83580.699
[2,   270] loss: 94475.450
[2,   300] loss: 102357.747
[2,   330] loss: 70864.239
Finished Training
